# ARIMA

- read data
- get only required (smooth)
- parse into 2 weeks and week
- transform
- train
- validate
- prediction model

*most part of code from Time series prediction using simple models.notebook*

### importing

In [2]:
import sys
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pandas import read_csv

### utils

In [3]:
def read_tsv_utf8(path):
    return read_csv(path, sep='\t', encoding='utf-8')

In [4]:
def date_from_str(str_date, format_date=''):
    if not format_date:
        format_date = "%Y-%m-%d"
    return datetime.datetime.strptime(str_date, format_date).date()

### read data

In [24]:
df = read_tsv_utf8("~/Documents/imagineML/data/time_series.tsv") 
columns = df.columns
df = df.drop([columns[0]], axis=1)
df['key_date'] = pd.to_datetime(df['key_date'], format='%Y-%m-%d')
df = df[df.columns[::-1]]
df = df.replace(np.nan, 0)
df.set_index('key_date', inplace=True)
df

,2428,4281,4681,3369,1739,5796,5795,715,2275,4670,...,4929,4822,2870,263,262,3523,3442,5388,4752,4911
key_date,,,,,,,,,,,,,,,,,,,,,
2014-07-01,0.0,1.0,5.0,3.0,7.0,2.0,2.0,0.0,1.0,2.0,...,7.0,4.0,46.0,53.0,19.0,5.0,17.0,13.000,10.0,17.0
2014-08-01,0.0,2.0,2.0,12.0,6.0,0.0,1.0,16.0,2.0,9.0,...,17.0,8.0,163.0,104.0,107.0,8.0,59.0,42.000,25.0,34.0
2014-09-01,2.0,1.0,2.0,34.0,12.0,0.0,0.0,12.0,3.0,10.0,...,12.0,2.0,249.0,80.0,129.0,8.0,35.0,27.000,56.0,38.0
2014-10-01,2.0,4.0,18.0,19.0,14.0,0.0,0.0,14.0,4.0,11.0,...,30.0,8.0,130.0,127.0,126.0,10.0,67.0,22.000,23.0,75.0
2014-11-01,3.0,2.0,10.0,22.0,2.0,0.0,0.0,5.0,3.0,5.0,...,24.0,8.0,151.0,98.0,81.0,7.0,52.0,32.000,26.0,79.0
2014-12-01,9.0,4.0,6.0,83.0,48.0,20.0,24.0,10.0,16.0,4.0,...,48.0,13.0,209.0,254.0,324.0,28.0,46.0,104.000,93.0,120.0
2015-01-01,5.0,2.0,15.0,24.0,14.0,1.0,2.0,5.0,3.0,9.0,...,9.0,6.0,147.0,67.0,85.0,7.0,32.0,14.000,39.0,46.0
2015-02-01,14.0,3.0,12.0,22.0,7.0,16.0,8.0,4.0,10.0,16.0,...,23.0,7.0,136.0,108.0,110.0,9.0,52.0,29.000,24.0,82.0
2015-03-01,2.0,2.0,3.0,23.0,10.0,6.0,15.0,5.0,7.0,10.0,...,20.0,6.0,90.0,79.0,65.0,6.0,27.0,31.000,21.0,22.0


### get only required (smooth)
ARIMA is appliable for smooth demand  
*data is received from Find most frequent sold items.notebook*

### parse data 2 weeks and week intervals
ARIMA requires 40 points at least  
*code from Forecasting.notebook*

In [53]:
from_date =  df.index.min()
till_date = df.index.max()
delta = till_date - from_date
print('from', from_date, 'till', till_date, 'in days', delta.days)

from 2014-07-01 00:00:00 till 2018-01-01 00:00:00 in days 1280


In [55]:
# months
sales_month = df.resample("M", how='sum')
print('from', from_date, 'till', till_date, 'in months', len(sales_month.index))
sales_month

from 2014-07-01 00:00:00 till 2018-01-01 00:00:00 in months 43


,2428,4281,4681,3369,1739,5796,5795,715,2275,4670,...,4929,4822,2870,263,262,3523,3442,5388,4752,4911
key_date,,,,,,,,,,,,,,,,,,,,,
2014-07-31,0.00000,1.00000,5.00000,3.00000,7.00000,2.00000,2.00000,0.00000,1.00000,2.00000,...,7.00000,4.00000,46.00000,53.00000,19.00000,5.00000,17.00000,13.00000,10.00000,17.00000
2014-08-31,0.00000,2.00000,2.00000,12.00000,6.00000,0.00000,1.00000,16.00000,2.00000,9.00000,...,17.00000,8.00000,163.00000,104.00000,107.00000,8.00000,59.00000,42.00000,25.00000,34.00000
2014-09-30,2.00000,1.00000,2.00000,34.00000,12.00000,0.00000,0.00000,12.00000,3.00000,10.00000,...,12.00000,2.00000,249.00000,80.00000,129.00000,8.00000,35.00000,27.00000,56.00000,38.00000
2014-10-31,2.00000,4.00000,18.00000,19.00000,14.00000,0.00000,0.00000,14.00000,4.00000,11.00000,...,30.00000,8.00000,130.00000,127.00000,126.00000,10.00000,67.00000,22.00000,23.00000,75.00000
2014-11-30,3.00000,2.00000,10.00000,22.00000,2.00000,0.00000,0.00000,5.00000,3.00000,5.00000,...,24.00000,8.00000,151.00000,98.00000,81.00000,7.00000,52.00000,32.00000,26.00000,79.00000
2014-12-31,9.00000,4.00000,6.00000,83.00000,48.00000,20.00000,24.00000,10.00000,16.00000,4.00000,...,48.00000,13.00000,209.00000,254.00000,324.00000,28.00000,46.00000,104.00000,93.00000,120.00000
2015-01-31,5.00000,2.00000,15.00000,24.00000,14.00000,1.00000,2.00000,5.00000,3.00000,9.00000,...,9.00000,6.00000,147.00000,67.00000,85.00000,7.00000,32.00000,14.00000,39.00000,46.00000
2015-02-28,14.00000,3.00000,12.00000,22.00000,7.00000,16.00000,8.00000,4.00000,10.00000,16.00000,...,23.00000,7.00000,136.00000,108.00000,110.00000,9.00000,52.00000,29.00000,24.00000,82.00000
2015-03-31,2.00000,2.00000,3.00000,23.00000,10.00000,6.00000,15.00000,5.00000,7.00000,10.00000,...,20.00000,6.00000,90.00000,79.00000,65.00000,6.00000,27.00000,31.00000,21.00000,22.00000


### stationarity

p-value > 0.05: Accept the null hypothesis (H0), the data has a unit root and is non-stationary.
p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

In [45]:
from statsmodels.tsa.stattools import adfuller
pd.options.display.float_format = '{:,.5f}'.format

def _color_red_or_green(val):
    if val <= 0.05:
        return "background-color: green"
    return ""

def stationarity_verfication(data):
    df_stationary = pd.DataFrame(columns= 
                                 ['item', 'ADF Statistic', 'p-value', 
                                  'Critical Values: 1%', 'Critical Values: 5%', 'Critical Values: 10%'])

    for item in data.columns:
        x = data[item]
        result = adfuller(x)
        critical_values = []
        for key, value in result[4].items():
            critical_values.append(value)
        values = [item, result[0], result[1]]
        values.extend(critical_values)
        df_stationary.loc[-1] = values  # adding a row
        df_stationary.index = df_stationary.index + 1  # shifting index
        df_stationary = df_stationary.sort_index()  # sorting by index 
    

    stationary_ts = df_stationary[df_stationary['p-value'] <= 0.05]['item']
    print('found stationary time series:', len(stationary_ts))
    print('non-stationary time series:', len(df_stationary) - len(stationary_ts))

    
    return df_stationary

In [51]:
sales_m_stationarity = stationarity_verfication(sales_month)
sales_m_stationarity.style.applymap(lambda x:_color_red_or_green(x), subset=['p-value'])

#len(sales_month)
len(df)

found stationary time series: 20
non-stationary time series: 14


43